This notebook pretends to use `fastai2` library to generate something similar as @iafoss did [here](https://www.kaggle.com/iafoss/panda-concat-tile-pooling-starter-0-79-lb). In order to do it, I reused part of his code together with this [notebook](https://www.kaggle.com/mnpinto/fastai2-working-with-3d-data/notebook) that uses fastai2 for working with "3D" images. For model training, I will use @drhabib's 2nd place model which can be found [here](https://github.com/DrHB/PANDA-2nd-place-solution) but only using the classification head.

In [ ]:
from fastai.vision.all import *

In [ ]:
TRAIN = '../input/panda-16x128x128-tiles-data/train/'
path = Path('../input/panda-16x128x128-tiles-data/train/')
#path_small = path[:100]


In [ ]:
LABELS = '../input/prostate-cancer-grade-assessment/train.csv'
df = pd.read_csv(LABELS)
df

In [ ]:
files = sorted(set([p for p in os.listdir(TRAIN)]))
files_id = [p[:32] for p in files]
df_2 = pd.DataFrame({'file': files, 'image_id': files_id})
result = pd.merge(df_2, df, on="image_id")

In [ ]:
result

In [ ]:
result['path'] = TRAIN + result['file']

In [ ]:
result

In [ ]:
result['isup_grade'].unique()

The following code uses the "3D" approach to build a `[bs x N x C x H X W]` Dataloader similar to the fastai1 version. In this case, however, it randomly sample a specific number of images. Moreover, in case some images does not have enough tiles (in this case n<12), it repeats some of them.

In [ ]:
def int2float(o:TensorImage):
    return o.float().div_(255.)

class ImageSequence(fastuple):
    @classmethod
    def create(cls, fns): return cls(tuple(PILImage.create(f) for f in fns))

def ImageSequenceBlock(): 
    return TransformBlock(type_tfms=ImageSequence.create, batch_tfms=int2float)

class SequenceGetItems():
    def __init__(self, filename_col, sequence_id_col, label_col, n_img):
        self.fn = filename_col
        self.seq = sequence_id_col
        self.label = label_col
        self.n_img = n_img
        
    def __call__(self, df):
        data = []
        for fn in progress_bar(df[self.seq].unique()):
            similar = df[self.seq] == fn
            similar = df.loc[similar]
            similar = similar.sample(self.n_img, replace=True)
            fns = similar[self.fn].tolist()
            lbl = similar[self.label].values[0]
            data.append([*fns, lbl])
        return data

def create_batch(data):
    xs, ys = [], []
    for d in data:
        xs.append(d[0])
        ys.append(d[1])
    xs = torch.cat([TensorImage(torch.cat([im[None] for im in x], dim=0))[None] for x in xs], dim=0)
    ys = torch.cat([y[None] for y in ys], dim=0)
    return TensorImage(xs), TensorCategory(ys)

def show_sequence_batch(max_n=2, n_img=10):
    xb, yb = dls.one_batch()
    fig, axes = plt.subplots(ncols=n_img, nrows=max_n, figsize=(12,6), dpi=120)
    for i in range(max_n):
        xs, ys = xb[i], yb[i]
        for j, x in enumerate(xs):
            axes[i,j].imshow(x.permute(1,2,0).cpu().numpy())
            axes[i,j].set_title(ys.item())
            axes[i,j].axis('off')

In [ ]:
n_img = 49
bs = 8

dblock = DataBlock(
    blocks    = (ImageSequenceBlock, CategoryBlock),
    get_items = SequenceGetItems('path', 'image_id', 'isup_grade', n_img), 
    get_x     = lambda t : t[:-1],
    get_y     = lambda t : t[-1],
    splitter  = RandomSplitter(valid_pct=0.2, seed=2020))

dls = dblock.dataloaders(result, bs=bs, create_batch=create_batch)
show_sequence_batch(n_img=n_img)

In [ ]:
dls.vocab

# Model

Here, I used the same model with some changes to fit the new `Dataloader`

In [ ]:
class CustomEnd(nn.Module):
    def __init__(self, scaler = SigmoidRange(-1, 6.0)):
        super().__init__()
        self.scaler_ = scaler
        
    def forward(self, x):
        classif = x[:, :-1]
        regress = self.scaler_ (x[:, -1])
        return classif, regress
    
def make_divisible(v, divisor=8, min_value=None):
    min_value = min_value or divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
   # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

def sigmoid(x, inplace: bool = False):
    return x.sigmoid_() if inplace else x.sigmoid()

class SqueezeExcite(nn.Module):
    def __init__(self, in_chs, se_ratio=0.25, reduced_base_chs=None,
             act_layer=nn.ReLU, gate_fn=sigmoid, divisor=1, **_):
        super(SqueezeExcite, self).__init__()
        self.gate_fn = gate_fn
        reduced_chs = make_divisible((reduced_base_chs or in_chs) * se_ratio, divisor)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_reduce = nn.Conv2d(in_chs, reduced_chs, 1, bias=True)
        self.act1 = act_layer(inplace=True)
        self.conv_expand = nn.Conv2d(reduced_chs, in_chs, 1, bias=True)
    def forward(self, x):
        x_se = self.avg_pool(x)
        x_se = self.conv_reduce(x_se)
        x_se = self.act1(x_se)
        x_se = self.conv_expand(x_se)
        x = x * self.gate_fn(x_se)
        return x

In [ ]:
class Model(nn.Module):
    def __init__(self, N=49):
        super().__init__()
        m = resnet34(pretrained=True)
        self.enc = nn.Sequential(*list(m.children())[:-2])       
        nc = list(m.children())[-1].in_features
        self.cb = SqueezeExcite(nc)
        self.head = nn.Sequential(AdaptiveConcatPool2d(),
                                  Flatten(),
                                  nn.Linear(2*nc,512),
                                  nn.ReLU(inplace=True),
                                  nn.Dropout(0.4),
                                  nn.Linear(512,6))
                                   
                                  #,CustomEnd())
        
        self.N=N
        
    def forward(self, x):
        shape = x.shape
        n = shape[1]
        x = x.view(-1,shape[2],shape[3],shape[4])
        x = self.enc(x)
        shape = x.shape
        x = x.view(-1, n, x.shape[1], x.shape[2], x.shape[3]).permute(0, 2, 1, 3, 4).contiguous().\
        view(-1, x.shape[1], x.shape[2] * n, x.shape[3])
        x = x.view(x.shape[0], x.shape[1], x.shape[2]//int(np.sqrt(self.N)), -1)
        x = self.cb(x)
        x = self.head(x)
        return x#[1]

In [ ]:
model = Model()

In [ ]:
learn = Learner(dls, model, metrics=[accuracy, CohenKappa(weights='quadratic')]).to_fp16()

In [ ]:
learn.fine_tune(5, freeze_epochs=5, cbs=[EarlyStoppingCallback(monitor='cohen_kappa_score', patience=5), 
                     SaveModelCallback(monitor='cohen_kappa_score', with_opt=True, fname='RN34SE_fastai2')])